In [17]:
import pandas as pd
import numpy as np

# 2009 Data

In [4]:
cdd_hdd = None
for i in range(1, 13):
    monthly_path = "2009/2009%02dmonthly.txt" % i
    station_path = "2009/2009%02dstation.txt" % i
    monthly = pd.read_csv(monthly_path, na_values='M')
    cdd_hdd_month = monthly[['WBAN', 'YearMonth', 'HeatingDegreeDays', 'CoolingDegreeDays']]
    if cdd_hdd is None:
        cdd_hdd = cdd_hdd_month
    else:
        cdd_hdd = pd.concat([cdd_hdd, cdd_hdd_month])
    stations = pd.read_csv(station_path, sep='|')

In [5]:
def valid_length(column):
    if len(column) == 12:
        return [1 for elem in column]
    else:
        return [0 for elem in column]
    
cdd_hdd = cdd_hdd[cdd_hdd.groupby('WBAN')['YearMonth'].transform(valid_length) == 1]

In [6]:
def valid_column(column):
    for elem in column:
        if np.isnan(elem):
            return [0 for e in column]
    return [1 for e in column]
    
cdd_hdd = cdd_hdd[cdd_hdd.groupby('WBAN')['CoolingDegreeDays'].transform(valid_column) == 1]
cdd_hdd = cdd_hdd[cdd_hdd.groupby('WBAN')['HeatingDegreeDays'].transform(valid_column) == 1]

In [7]:
station_cdd_hdd = cdd_hdd.groupby('WBAN')[['HeatingDegreeDays', 'CoolingDegreeDays']].aggregate(np.sum)

# 30 Year Data

In [67]:
separations = [0, 7, 27]
separations += np.arange(30, 114, 6).tolist()
separations += [114]

rows = []
with open('30yr.txt') as f:
    for line in f.readlines():
        line = line.strip()
        row = []
        for i in range(len(separations) - 1):
            row.append(line[separations[i]:separations[i+1]].strip())
        rows.append(row)

In [68]:
climate_norm = pd.DataFrame(rows, columns=['COOP', 'Name', 'State',
                            'HDD65', 'HDD60', 'HDD57',
                            'HDD55', 'HDD50', 'HDD45', 'HDD40',
                            'CDD70', 'CDD65', 'CDD60', 'CDD57',
                            'CDD55', 'CDD50', 'CDD45'])

In [69]:
climate_norm.to_csv('30yr.csv',index=False)

In [70]:
separations
rows = []
with open('emshr_lite.txt', encoding='latin1') as f:
    headers = f.readline().strip().split()
    loc_str = f.readline().strip()
    separations = np.where(np.array([0 if c == '-' else 1 for c in loc_str]))[0].tolist()
    separations = [0] + separations + [len(loc_str)]
    
    for line in f.readlines():
        line = line.strip()
        row = []
        for i in range(len(separations) - 1):
            row.append(line[separations[i]:separations[i+1]].strip())
        rows.append(row)

In [71]:
stations = pd.DataFrame(rows, columns=headers)

In [72]:
stations.to_csv('station_metadata.csv', index=False)

# Normals 1981-2010

   Name      | Position | Type
   ----------|----------|-----------
     STNID   |    1- 11 | Character 
     VALUE   |  19- 23  | Integer   
     FLAG    |  24- 24  | Character 

In [51]:
rows = []
with open('ann-htdd-normal.txt') as f:
    for line in f.readlines():
        station, value_flag = line.strip().split()
        value = int(value_flag[:-1])
        if value < 0:
            value = np.nan
        flag = value_flag[-1]
        row = [station, value, flag]
        rows.append(row)
hdd = pd.DataFrame(rows, columns=['StationId', 'HDD30', 'HDD30Flag'])

rows = []
with open('ann-cldd-normal.txt') as f:
    for line in f.readlines():
        station, value_flag = line.strip().split()
        value = int(value_flag[:-1])
        if value < 0:
            value = np.nan
        flag = value_flag[-1]
        row = [station, value, flag]
        rows.append(row)
cdd = pd.DataFrame(rows, columns=['StationId', 'CDD30', 'CDD30Flag'])

cdd_hdd = pd.merge(hdd, cdd, on='StationId')
cdd_hdd.dropna(inplace=True)

       
      Name       | Pos     | Type
      -----------|---------|-------------
       ID        |    1-11 |  Character
       LATITUDE  |   13-20 |  Real
       LONGITUDE |   22-30 |  Real
       ELEVATION |   32-37 |  Real
       STATE     |   39-40 |  Character
       NAME      |   42-71 |  Character
       GSNFLAG   |   73-75 |  Character
       HCNFLAG   |   77-79 |  Character
       WMOID     |   81-85 |  Character
       METHOD 	 |   87-99 |  Character
       

In [52]:
rows = []
with open('temp-inventory.txt') as f:
    for line in f.readlines():
        line = line.strip()
        station = line[:11]
        lat = float(line[12:20])
        lng = float(line[21:30])
        rows.append([station, lat, lng])
stations = pd.DataFrame(rows, columns=['StationId', 'Lat', 'Lng'])

In [53]:
cdd_hdd_loc = pd.merge(cdd_hdd, stations, on='StationId')

In [55]:
cdd_hdd_loc[['Lat', 'Lng', 'CDD30', 'HDD30']].to_csv('30yr_with_location_recent.csv', index=False)

# Merge

In [73]:
climate_norm = pd.read_csv('30yr.csv')
stations = pd.read_csv('station_metadata.csv')

In [74]:
lat_lng_cdd_hdd = pd.merge(stations, climate_norm, on=['COOP']).groupby('COOP').nth(0)[['LAT_DEC', 'LON_DEC', 'HDD65', 'CDD65']]

In [75]:
lat_lng_cdd_hdd.rename(columns={'LAT_DEC': 'Lat', 'LON_DEC': 'Lng', 'HDD65': 'HDD30', 'CDD65': 'CDD30'}, inplace=True)

In [76]:
lat_lng_cdd_hdd.to_csv('30yr_with_location.csv', index=False)